# Esercizio 3 - Hanks

- Scegliere un verbo transitivo --> **KILL**
  
- Trovare un corpus in cui comprare il verbo scelto --> https://sentence.yourdictionary.com/kill
- Effettuare parsing e disambiguazione
- Usare i supersensi di wordnet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Calcolo risultati, frequenza e stampare cluster semantici ottenuti

### Approccio

- Si parte dal daatset du frasi col verbo *kill*, si estraggono tramite il *Dependency Matcher* di *Spacy* oggetto e sggetto del verbo. Si ricavano poi i synset di entrambi. 

- Visto che il dataset presenta molte frasi in cui soggeto o oggetto sono parole come *"you", "me", "someone",...* (lista completa in person).

- Viene automaticamente associato il synset *person* a tutti i termini simili a quelli precentemente citati.

- Infine vengono stampate le statistiche, indicando la percentuale di volte in cui il verbo kill compare con la coppia (soggetto, oggetto) appartenenti alla stessa categoria.

In [15]:
from nltk.corpus import wordnet
from spacy.matcher import DependencyMatcher
from nltk.wsd import lesk
import re
import spacy

* person: pronomi riconducibili al synset person
* pattern: usati per trovare il soggetto e l'oggetto del verbo kill

In [16]:
person = ["i", "you", "he", "she", "we", "they", "me", "him", "her", "his", "them", "someone", "us", "people", "anyone"] 

pattern1 = [
    {"RIGHT_ID": "attr",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["kill"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

pattern2 = [
    {"RIGHT_ID": "verb",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["want", "wish"]}}
    },
    {"LEFT_ID": "verb",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "verb",
    "REL_OP": ">",
    "RIGHT_ID": "xcomp",
    "RIGHT_ATTRS": {"DEP": {"IN": ["xcomp"]}}
    },
    {"LEFT_ID": "xcomp",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

In [17]:
nlp = spacy.load('en_core_web_sm')

matcher = DependencyMatcher(nlp.vocab)
matcher.add("pattern1", [pattern1])
matcher.add("pattern2", [pattern2])

#### Metodi per trovare il soggetto e l'oggetto del verbo e per fare word sense disambiguation

In [18]:
def get_match(text): # individua il pattern nel testo
    doc = nlp(text)
    matches = matcher(doc)
    for match in matches:
        match_words = sorted(match[1])
        phrase = doc[match_words[0]:match_words[len(match_words)-1]+1]
        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text
        
        return subj, dobj, phrase[0].tag_, phrase[len(phrase)-1].tag_
    return "","","",""

def word_sense_disambiguation(list_words, word): 
    right_synset = lesk(list_words, word)
    return right_synset

In [19]:
def cleaner(text):
    return text.split('.')[1]

In [20]:
subj_ss = ""
dobj_ss = ""
struct = {}
tot = 0
# counter = 0
with open ('../data/sentence_kill.txt', 'r', encoding="utf8") as f:
    for row in f:
        subj_synset, dobj_synset, subj_ss, dobj_ss = None, None, "", ""
        subj, dobj, stag, dtag = get_match(row)
            
        if subj != "" and dobj != "":
            # Cerco il synset del soggetto e dell'oggetto e associo automaticamente il synset "person" se trovo
            # un nome proprio o una sringa presente in person
            
            # Soggetto
            if stag == "NNP" or subj.lower() in person:
                subj_ss = "person"
            else:
                subj_synset = word_sense_disambiguation(re.findall(r'\w+', row), subj)
                
            # Oggetto
            if dtag == "NNP" or dobj.lower() in person:
                dobj_ss = "person"
            else:
                dobj_synset = word_sense_disambiguation(re.findall(r'\w+', row), dobj)        
            
            # Soggetto - Se subj_synset e' None, significa che abbiamo associato il synset person
            if not subj_synset is None:
                subj_ss = cleaner(subj_synset.lexname()) # supersense
            elif subj_ss != "person":
                subj_ss = "unknown"
            
            # Oggetto
            if not dobj_synset is None:
                dobj_ss = cleaner(dobj_synset.lexname())  
            elif dobj_ss != "person":
                dobj_ss = "unknown"

            if (subj_ss, dobj_ss) in struct:
                struct[(subj_ss, dobj_ss)] += 1
            else:
                struct[(subj_ss, dobj_ss)] = 1
        '''else:
            counter+=1
            print(f"phrase {counter}: {row}")'''

# usato per controllare il numero di frasi senza match
for (k,v) in zip(struct.keys(),struct.values()):
    if (k[0] != "unknown" and k[1] != "unknown") and v > 1:
        tot += v

somma_tot = 0
for k in struct.keys():
    if (k[0] != "unknown" and k[1] != "unknown") and struct[k] > 1:
        print(f"{k}: {round(((struct[k]/tot)*100), 2)} %")
        somma_tot += ((struct[k]/tot)*100)

print(f"Somma delle percentuali: {round(somma_tot, 3)} %")


('artifact', 'communication'): 1.16 %
('person', 'person'): 70.52 %
('person', 'group'): 1.73 %
('person', 'communication'): 2.89 %
('person', 'animal'): 3.47 %
('act', 'person'): 1.16 %
('person', 'all'): 5.2 %
('person', 'artifact'): 2.31 %
('cognition', 'person'): 1.73 %
('person', 'cognition'): 1.16 %
('person', 'act'): 1.73 %
('group', 'person'): 3.47 %
('contact', 'person'): 1.16 %
('person', 'emotion'): 1.16 %
('social', 'person'): 1.16 %
Somma delle percentuali: 100.0 %


### Analisi dei risultati

Le percentuali ottenute indicano come il verbo *kill* sia usato principalmente con person come soggetto e oggetto (70% dei casi).

Il *dependency matcher* di *Spacy* rende l'approccio scalabile, in quanto permette di indivudare elementi sintattici in modo semplice e veloce. 

Problemi riscontrati:

- *WSD*: la funzioine usata è l'algoritmo di Lesk, che funziona nel 50-60% dei casi;
- *Dataset*: il dataset usato è piccolo, e contiene frasi poco varie;